In [1]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

### Choose a time partition

In [62]:
tp = 4

Load Database

In [63]:
data  = pd.read_csv(f'datos\counted\d{tp}_counted.csv', index_col=0)

In [64]:
data.columns

Index(['avisoid', 'empresaid', 'avisofechapublicacion', 'avisovacante',
       'mostrarsueldo', 'avisoexperiencia', 'expiracion', 'dias',
       'avisorepublicacion', 'avisocargo', 'areanombre',
       'actividadempresanombre', 'avisocuerpo', 'disponibilidadnombre',
       'avisoduracioncont', 'avisolugartrabajo', 'gradoescolarnombre',
       'situacionestudios', 'avisorequisitos', 'carreras', 'estado',
       'endpagado_o_gratuito', 'nivelnombre', 'carreras_array',
       'sueldoestimado', '_merge', 'teletrabajo_', 'avcu_b1', 'avca_b1',
       'avre_b1', 'avcu_b2', 'avca_b2', 'avre_b2', 'avcu_b3', 'avca_b3',
       'avre_b3', 'avcu_b4', 'avca_b4', 'avre_b4', 'bucket1', 'bucket2',
       'bucket3', 'bucket4', 'wfh'],
      dtype='object')

In [65]:
data['avisofechapublicacion'] = pd.to_datetime(data['avisofechapublicacion'])

### Choose a window size in days

In [66]:
days_before = 45
days_after = 45

In [67]:
dates = data['avisofechapublicacion'].unique()

In [68]:
dates = pd.Series(data['avisofechapublicacion'].unique())

avisocuerpo, empresaid, avisolugartrabajo

In [70]:
data['duplicates_avisocuerpo'] = False

In [71]:
window_before = timedelta(days=days_before)
window_after = timedelta(days=days_after)

for x in dates.index:
    start_date = dates[x] - window_before
    final_date = dates[x] + window_after

    # Sub set with the date window
    newData = data[(data['avisofechapublicacion'] >= start_date) & (data['avisofechapublicacion'] < final_date)]

    # Another Data frame with the duplicates for the newData subset
    duplicates = newData[newData.duplicated(subset=['empresaid', 'avisocuerpo', 'avisolugartrabajo'], keep='first')]  # Dont consider the first realiztion, mark everything else as duplicate

    # Check wich ad from Data dataFrame is in the duplicates dataFrame
    data['duplicates_avisocuerpo'] = data['duplicates_avisocuerpo'] + data.index.isin(duplicates.index)


avisocargo, avisocuerpo, empresaid, avisolugartrabajo

In [72]:
data['duplicates_avisocargo'] = False

In [73]:
window_before = timedelta(days=days_before)
window_after = timedelta(days=days_after)

for x in dates.index:
    start_date = dates[x] - window_before
    final_date = dates[x] + window_after

    # Sub set with the date window
    newData = data[(data['avisofechapublicacion'] >= start_date) & (data['avisofechapublicacion'] < final_date)]

    # Another Data frame with the duplicates for the newData subset
    duplicates = newData[newData.duplicated(subset=['empresaid', 'avisocargo', 'avisocuerpo', 'avisolugartrabajo'], keep='first')]  # Dont consider the first realiztion, mark everything else as duplicate

    # Check wich ad from Data dataFrame is in the duplicates dataFrame
    data['duplicates_avisocargo'] = data['duplicates_avisocargo'] + data.index.isin(duplicates.index)

Save the data

In [74]:
data.to_csv(f'datos\counted\d{tp}_counted.csv')